## Sweep bot

i'm fucking crashing out
why is nothing fucking working

In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    ConstantVectorSource,
    ContactModel,
    Demultiplexer,
    DiagramBuilder,
    InverseDynamicsController,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import (
    AddMeshcatTriad,
    PublishPositionTrajectory,
)
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario
from scripts.grasp_broom import plan_path, grasp_path, build_temp_plant
from scripts.meta_controller import MetaController

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


In [ ]:
from scripts.final_hail_mary import plan_sweep
use_hydroelastic = True # this takes forever

def make_diagram():

    scenario = load_scenario(use_position=True, use_weld=True)

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            # one object rigid, all else compliant? table
            # HE modulus parameters
            # allow broom to turn more easily

            # add "handle" to broom to make it easier to grip?
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)
        # AddDefaultVisualization(parser.builder(), meshcat)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    plant: MultibodyPlant = station.plant()

    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    broom_frame = plant.GetFrameByName("handle_link")

    trajectory = plan_sweep(plant, plant_context, broom_frame, X_WStart, X_WGoal)


    wsg_src = builder.AddSystem(ConstantVectorSource([0.5]))
    builder.Connect(
        meta_controller.get_output_port(),
        demux.get_input_port(),
    )

    # builder.Connect(
    #     demux.get_output_port(0),
    #     control_desired_state_input_port
    # )

    builder.Connect(
        demux.get_output_port(0),
        station.GetInputPort("iiwa.position")
    )
    builder.ExportOutput(demux.get_output_port(0), 'pos')
    builder.ExportOutput(demux.get_output_port(1), 'vel')

    builder.Connect(
        wsg_src.get_output_port(),
        station.GetInputPort("wsg.position")
    )

    # builder.Connect(
    #     station.GetOutputPort("iiwa_state"),
    #     control_estimated_state_input_port
    # )

    # builder.Connect(
    #     control_output_port,
    #     station.GetInputPort("iiwa_actuation")
    # )
    # AddDefaultVisualization(builder, meshcat)

    diagram = builder.Build()
    # meta_controller_sys.add_diagram(diagram)

    # for debugging
    meta_controller_sys.meshcat = meshcat

    return diagram, plant, station, meta_controller_sys

# diagram = builder.Build()

In [5]:
diagram, plant, station, controller = make_diagram()

In [8]:

simulator = Simulator(diagram)
context = simulator.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(context)
controller.add_diagram(diagram, context, meshcat)
iiwa = plant.GetModelInstanceByName('iiwa')
# broom = plant.GetModelInstanceByName('broom')
# iiwa_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("base", iiwa))
# broom_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("handle_link"))
# AddMeshcatTriad(meshcat, 'iiwa', X_PT=iiwa_pose)
# AddMeshcatTriad(meshcat, 'broom', X_PT=broom_pose)
from scripts.trajectory_helpers import MoveToStart
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
end_time = 3.4
time = 0

traj_gen = MoveToStart()

pos_port = simulator.get_system().GetOutputPort('pos')
# pant.SetPOsitions(pant_context, trajectory.value(t))
#                 visaulizer.ForcedPublish(visaulzier_context)
# trajectory, _ = traj_gen.trajectory(controller)
simulator.AdvanceTo(0.001)


[[-0.13621077 -0.13463403 -0.1330578  ... -0.04846403 -0.04647926
  -0.04449448]
 [ 0.8001246   0.8001006   0.80007666 ...  0.28342864  0.28342864
   0.28342864]
 [ 0.26109767  0.26120996  0.26132256 ...  0.0499999   0.0499999
   0.0499999 ]]
empty trajectories at time 0.0
called at time 0.0
start!
7
(7, 10)
Solver failed
['path position constraint at s=0[0]: 0.014904 <= 0.014059 <= 0.014904', 'path position constraint at s=0[1]: 0.719573 <= 0.720164 <= 0.719573', 'path position constraint at s=0[2]: 0.223269 <= 0.223511 <= 0.223269', 'path position constraint at s=1[0]: 2.999999 <= 2.969556 <= 3.000000', 'path position constraint at s=0[0]: -inf <= 2.480758 <= 1.000000', 'path position constraint at s=0.7083333333333333[0]: -inf <= 469.656904 <= 1.000000', 'path position constraint at s=0.75[0]: -inf <= 13.975641 <= 1.000000', 'path position constraint at s=0.9166666666666666[0]: -inf <= 1.382152 <= 1.000000']
attempts needed: 1
[1.0, 0.0, 0.0, 0.0]
will end at: 3.197578433494916


In [ ]:

while time < end_time:
    time += 0.1
    # plant.SetPositions()
    print(time)
    print(pos_port.Eval(context))
    plant.SetPositions(plant_context, iiwa, pos_port.Eval(context))
    simulator.AdvanceTo(time)
# simulator.AdvanceTo(5)

meshcat.StopRecording()
meshcat.PublishRecording()
# controller.sub_meshcat.StopRecording()
# controller.sub_meshcat.PublishRecording()

0.1
[-1.09190177  1.57796066 -0.99499426  1.15096026 -0.09100423  1.74259161
 -0.10829876]
0.2
[-1.06869969  1.56068758 -0.98141085  1.1215861  -0.05149005  1.70003862
 -0.07702308]
0.30000000000000004
[-1.01158905  1.51077265 -0.95278774  1.04658869  0.0402555   1.59259339
  0.00278206]
0.4
[-0.94076837  1.43206245 -0.92822809  0.94746394  0.14148943  1.45321124
  0.10815786]
0.5
[-0.87617162  1.32822356 -0.92666915  0.84537897  0.20993191  1.31436845
  0.21649568]
0.6
[-0.83743327  1.20299807 -0.96659654  0.76099721  0.2042623   1.20793994
  0.30559057]
0.7
[-0.83447972  1.06257668 -1.05185543  0.69865366  0.11425394  1.14630619
  0.36632415]
0.7999999999999999
[-0.86564318  0.91607442 -1.16880611  0.64318253 -0.03318468  1.11856573
  0.40520706]


KeyboardInterrupt: 